<h3>Hyperopt Tuning </h3>
Could add early stopping with a validation set and return it back like this code:
https://www.kaggle.com/inspector/keras-hyperopt-example-sketch

and I think this shows how to get the number of rounds from https://stackoverflow.com/questions/49852241/return-number-of-epochs-for-earlystopping-callback-in-keras
where I guess you would not use kfold and instead train on full train with a dedicated validation set


In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import hyperopt.pyll.stochastic
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold,ParameterGrid
from sklearn.metrics import mean_squared_error 
from keras import Sequential
from keras.layers import Dense
import time
import datetime


t0= time.clock()

boston = load_boston()
y=boston.target
X=boston.data


space = {
    'dense_1': hp.quniform('dense_1', 5,10,1),
    'optim': hp.choice('optim', ['adam', 'rmsprop']),
    'batch_size': hp.choice('batch_size', [8,16,32]),
    'epochs': hp.choice('epochs', [1,2,3,4]),
    
    'num_layers' :hp.choice('num_layers',
    [
                    {'layers':'one',
                    
                    },
        
                    {'layers':'two',
                      
                        'dense_2': hp.quniform('dense_2', 5,10,1)
                                
                    }

    ])
    
    
}



kf=KFold(n_splits=3,random_state=4521)

def build_model (params):
    model = Sequential()
    model.add(Dense(int(params['dense_1']), input_dim=13, kernel_initializer='normal', activation='relu'))
    
    if params['num_layers'] == 'two':
        model.add(Dense(int(params['dense_2']), kernel_initializer='normal', activation='relu'))
      
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=params['optim'])
    return (model)

    
def obj_function (params):
    model=build_model(params)
    
    hold_actual=np.zeros((X.shape[0],1))
    hold_preds=np.zeros((X.shape[0],1))

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train,y_train,batch_size=params['batch_size'], epochs=params['epochs'])
        pred=model.predict(X_test)
        hold_actual[test_index,0]=y_test.ravel()
        hold_preds[test_index,0]=pred.ravel()

    return(mean_squared_error(hold_actual,hold_preds))
    
trials = Trials()
best = fmin(obj_function, space, algo=tpe.suggest, max_evals=50, trials=trials)
print (best)

t1 = time.clock() - t0

str(datetime.timedelta(seconds=t1))

Epoch 1/3
337/337 [==============================] - 1s 4ms/step - loss: 544.4332
Epoch 2/3
337/337 [==============================] - 0s 69us/step - loss: 454.0971
Epoch 3/3
337/337 [==============================] - 0s 79us/step - loss: 375.3731
Epoch 1/3
337/337 [==============================] - 0s 67us/step - loss: 169.1701
Epoch 2/3
337/337 [==============================] - 0s 68us/step - loss: 141.6657
Epoch 3/3
337/337 [==============================] - 0s 75us/step - loss: 125.2757
Epoch 1/3
338/338 [==============================] - 0s 70us/step - loss: 205.7352
Epoch 2/3
338/338 [==============================] - 0s 72us/step - loss: 141.6986
Epoch 3/3
338/338 [==============================] - 0s 82us/step - loss: 111.9165
Epoch 1/1
337/337 [==============================] - 1s 2ms/step - loss: 638.6276
Epoch 1/1
337/337 [==============================] - 0s 253us/step - loss: 188.8079
Epoch 1/1
338/338 [==============================] - 0s 249us/step - loss: 150.9365
Epoc

338/338 [==============================] - 0s 133us/step - loss: 83.4485
Epoch 1/3
337/337 [==============================] - 1s 3ms/step - loss: 642.3632
Epoch 2/3
337/337 [==============================] - 0s 203us/step - loss: 389.3749
Epoch 3/3
337/337 [==============================] - 0s 182us/step - loss: 201.3957
Epoch 1/3
337/337 [==============================] - 0s 176us/step - loss: 86.7819
Epoch 2/3
337/337 [==============================] - 0s 170us/step - loss: 78.2535
Epoch 3/3
337/337 [==============================] - 0s 153us/step - loss: 75.4705
Epoch 1/3
338/338 [==============================] - 0s 169us/step - loss: 109.9598
Epoch 2/3
338/338 [==============================] - 0s 171us/step - loss: 78.9196
Epoch 3/3
338/338 [==============================] - 0s 168us/step - loss: 77.1360
Epoch 1/1
337/337 [==============================] - 1s 3ms/step - loss: 710.0507
Epoch 1/1
337/337 [==============================] - 0s 141us/step - loss: 353.6256
Epoch 1/1
33

337/337 [==============================] - 0s 83us/step - loss: 456.5506
Epoch 3/4
337/337 [==============================] - 0s 81us/step - loss: 359.3975
Epoch 4/4
337/337 [==============================] - 0s 88us/step - loss: 267.5388
Epoch 1/4
337/337 [==============================] - 0s 76us/step - loss: 102.4244
Epoch 2/4
337/337 [==============================] - 0s 74us/step - loss: 89.7809
Epoch 3/4
337/337 [==============================] - 0s 86us/step - loss: 88.8796
Epoch 4/4
337/337 [==============================] - 0s 78us/step - loss: 86.8100
Epoch 1/4
338/338 [==============================] - 0s 79us/step - loss: 144.0027
Epoch 2/4
338/338 [==============================] - 0s 87us/step - loss: 103.0397
Epoch 3/4
338/338 [==============================] - 0s 87us/step - loss: 84.0765
Epoch 4/4
338/338 [==============================] - 0s 83us/step - loss: 82.5733
Epoch 1/4
337/337 [==============================] - 1s 4ms/step - loss: 534.7409
Epoch 2/4
337/337 [=

337/337 [==============================] - 0s 264us/step - loss: 99.7971
Epoch 1/4
337/337 [==============================] - 0s 330us/step - loss: 61.9666
Epoch 2/4
337/337 [==============================] - 0s 275us/step - loss: 60.6369
Epoch 3/4
337/337 [==============================] - 0s 266us/step - loss: 57.4482
Epoch 4/4
337/337 [==============================] - 0s 276us/step - loss: 54.8535
Epoch 1/4
338/338 [==============================] - 0s 271us/step - loss: 63.9845
Epoch 2/4
338/338 [==============================] - 0s 263us/step - loss: 62.1325
Epoch 3/4
338/338 [==============================] - 0s 265us/step - loss: 61.7884
Epoch 4/4
338/338 [==============================] - 0s 267us/step - loss: 60.1690
Epoch 1/2
337/337 [==============================] - 1s 4ms/step - loss: 403.8381
Epoch 2/2
337/337 [==============================] - 0s 239us/step - loss: 147.8415
Epoch 1/2
337/337 [==============================] - 0s 293us/step - loss: 69.6544
Epoch 2/2
337/

338/338 [==============================] - 0s 139us/step - loss: 74.3886
Epoch 2/4
338/338 [==============================] - 0s 146us/step - loss: 67.4480
Epoch 3/4
338/338 [==============================] - 0s 140us/step - loss: 65.5598
Epoch 4/4
338/338 [==============================] - 0s 143us/step - loss: 64.1212
Epoch 1/1
337/337 [==============================] - 1s 4ms/step - loss: 334.3169
Epoch 1/1
337/337 [==============================] - 0s 267us/step - loss: 75.9937
Epoch 1/1
338/338 [==============================] - 0s 307us/step - loss: 86.7220
Epoch 1/4
337/337 [==============================] - 1s 4ms/step - loss: 518.5189
Epoch 2/4
337/337 [==============================] - 0s 140us/step - loss: 223.0280
Epoch 3/4
337/337 [==============================] - 0s 148us/step - loss: 149.4459
Epoch 4/4
337/337 [==============================] - 0s 145us/step - loss: 133.5405
Epoch 1/4
337/337 [==============================] - 0s 143us/step - loss: 72.0448
Epoch 2/4
337

'0:02:32.629263'

In [ ]:
trials.trials[4]['result']

<h3>RAY</h3>

In [1]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import ray
from ray.tune import run_experiments, register_trainable
from ray.tune.hpo_scheduler import HyperOptScheduler

import hyperopt.pyll.stochastic
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold,ParameterGrid
from sklearn.metrics import mean_squared_error 
from keras import Sequential
from keras.layers import Dense

import time
import datetime


t0= time.clock()





boston = load_boston()
y=boston.target
X=boston.data


space = {
    'dense_1': hp.quniform('dense_1', 5,10,1),
    'optim': hp.choice('optim', ['adam', 'rmsprop']),
    'batch_size': hp.choice('batch_size', [8,16,32]),
    'epochs': hp.choice('epochs', [1,2,3,4]),
    
    'num_layers' :hp.choice('num_layers',
    [
                    {'layers':'one',
                    
                    },
        
                    {'layers':'two',
                      
                        'dense_2': hp.quniform('dense_2', 5,10,1)
                                
                    }

    ])
    
    
}



kf=KFold(n_splits=3,random_state=4521)

def build_model (config):
    model = Sequential()
    model.add(Dense(int(config['dense_1']), input_dim=13, kernel_initializer='normal', activation='relu'))
    
    if config['num_layers'] == 'two':
        model.add(Dense(int(config['dense_2']), kernel_initializer='normal', activation='relu'))
      
    model.add(Dense(1, kernel_initializer='normal'))
    model.compile(loss='mean_squared_error', optimizer=config['optim'])
    return (model)

    
def obj_function (config, reporter):
    model=build_model(config)
    
    hold_actual=np.zeros((X.shape[0],1))
    hold_preds=np.zeros((X.shape[0],1))

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.fit(X_train,y_train,batch_size=config['batch_size'], epochs=config['epochs'])
        pred=model.predict(X_test)
        hold_actual[test_index,0]=y_test.ravel()
        hold_preds[test_index,0]=pred.ravel()
        
    mse=mean_squared_error(hold_actual,hold_preds)
    reporter(timesteps_total=1, mean_loss=mse)
    

    
config = {
        "my_exp": {
            "run": "exp", 
            
            "stop": {
                "timesteps_total": 1
            },
            "trial_resources": { "cpu": 8, "gpu": 0 },
            "repeat": 50, #this seems to overide the timesteps_total stop, else it would only run 1 time given above stop setting
            "config": {
                "space": space
            }
        }
    }

ray.init()
register_trainable("exp", obj_function) #registers the above config and the objective function

# "the HyperOptScheduler will take an increasing metric as it's "reward metric", 
#similar to all other schedulers. 
#However, since we want loss to "decrease" = we want neg loss to "increase", 
#we pass "neg loss" to the reward metric."
#so, all schedulers take an increasing metric (meaning they look for higher = better)
#negating it allows for preferring a decreasing metric (as a lower loss -> higher neg_loss)

hpo_sched = HyperOptScheduler(reward_attr="neg_mean_loss")

trials=run_experiments(config, verbose=False, scheduler=hpo_sched)


t1 = time.clock() - t0

str(datetime.timedelta(seconds=t1))


/home/jma/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/jma/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:48470 to respond...
Waiting for redis server at 127.0.0.1:63739 to respond...
Starting local scheduler with the following resources: {'CPU': 8, 'GPU': 1}.

View the web UI at http://localhost:8895/notebooks/ray_ui39983.ipynb?token=0e683471f48dabc4ccd3e699b683099b0ec02c832bc89908



== Status ==
Using FIFO scheduling algorithm.
Result logdir: /home/jma/ray_results/my_exp
PENDING trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	PENDING

Created LogSyncer for /home/jma/ray_results/my_exp/exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam_2018-07-19_15-45-23em6ooaka -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 8/8 CPUs, 1/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
RUNNING trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	RUNNING

Created LogSyncer for /home/jma/ray_results/my_exp/exp_1_batch_size=16_dense_1=7.0_epochs=3_num_layers={'lay_optim=adam_2018-07-19_15-45-28byf39h8b -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
PENDING trials:
 - exp_2_batch_size=16_dense_1=7.0_epochs=1_num_layers={'lay_optim=rmspr:	PENDING
TERMINATED trials:
 - exp_0_batch_size=8_dense_1=7.0_ep

Created LogSyncer for /home/jma/ray_results/my_exp/exp_15_batch_size=16_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr_2018-07-19_15-46-04mlo8qqve -> 
Created LogSyncer for /home/jma/ray_results/my_exp/exp_16_batch_size=16_dense_1=5.0_epochs=3_num_layers={'den_optim=adam_2018-07-19_15-46-07_3wkozw1 -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
PENDING trials:
 - exp_17_batch_size=16_dense_1=7.0_epochs=2_num_layers={'lay_optim=adam:	PENDING
TERMINATED trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25521], 2 s, 1 ts, 412 loss
 - exp_10_batch_size=8_dense_1=6.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25966], 2 s, 1 ts, 258 loss
 - exp_11_batch_size=8_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26004], 2 s, 1 ts, 88.5 loss
 - exp_12_batch_size=8_dense_1=5.0_epochs=4_num_layers={'den_optim=adam:	TERMINATED [pid=260

Created LogSyncer for /home/jma/ray_results/my_exp/exp_22_batch_size=16_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr_2018-07-19_15-46-23dmrj2jco -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
PENDING trials:
 - exp_23_batch_size=32_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	PENDING
TERMINATED trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25521], 2 s, 1 ts, 412 loss
 - exp_10_batch_size=8_dense_1=6.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25966], 2 s, 1 ts, 258 loss
 - exp_11_batch_size=8_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26004], 2 s, 1 ts, 88.5 loss
 - exp_12_batch_size=8_dense_1=5.0_epochs=4_num_layers={'den_optim=adam:	TERMINATED [pid=26042], 2 s, 1 ts, 97.3 loss
 - exp_13_batch_size=16_dense_1=8.0_epochs=3_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26080], 2 s, 1 ts, 171 loss
 - exp

Created LogSyncer for /home/jma/ray_results/my_exp/exp_28_batch_size=16_dense_1=10.0_epochs=1_num_layers={'lay_optim=rmspr_2018-07-19_15-46-409lg5xvnz -> 
Created LogSyncer for /home/jma/ray_results/my_exp/exp_29_batch_size=16_dense_1=9.0_epochs=4_num_layers={'lay_optim=rmspr_2018-07-19_15-46-41lc4z0jhp -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
PENDING trials:
 - exp_30_batch_size=16_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	PENDING
TERMINATED trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25521], 2 s, 1 ts, 412 loss
 - exp_10_batch_size=8_dense_1=6.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25966], 2 s, 1 ts, 258 loss
 - exp_11_batch_size=8_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26004], 2 s, 1 ts, 88.5 loss
 - exp_12_batch_size=8_dense_1=5.0_epochs=4_num_layers={'den_optim=adam:	TERMINATED [pid=

Created LogSyncer for /home/jma/ray_results/my_exp/exp_36_batch_size=16_dense_1=10.0_epochs=1_num_layers={'lay_optim=rmspr_2018-07-19_15-46-59vwrti41s -> 
Created LogSyncer for /home/jma/ray_results/my_exp/exp_37_batch_size=8_dense_1=7.0_epochs=2_num_layers={'lay_optim=rmspr_2018-07-19_15-47-01i7yhut9e -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
PENDING trials:
 - exp_38_batch_size=16_dense_1=8.0_epochs=4_num_layers={'lay_optim=rmspr:	PENDING
TERMINATED trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25521], 2 s, 1 ts, 412 loss
 - exp_10_batch_size=8_dense_1=6.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25966], 2 s, 1 ts, 258 loss
 - exp_11_batch_size=8_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26004], 2 s, 1 ts, 88.5 loss
 - exp_12_batch_size=8_dense_1=5.0_epochs=4_num_layers={'den_optim=adam:	TERMINATED [pid=26

Created LogSyncer for /home/jma/ray_results/my_exp/exp_43_batch_size=16_dense_1=9.0_epochs=4_num_layers={'den_optim=adam_2018-07-19_15-47-17ss4bbqmx -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
PENDING trials:
 - exp_44_batch_size=16_dense_1=8.0_epochs=4_num_layers={'lay_optim=rmspr:	PENDING
TERMINATED trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25521], 2 s, 1 ts, 412 loss
 - exp_10_batch_size=8_dense_1=6.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25966], 2 s, 1 ts, 258 loss
 - exp_11_batch_size=8_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26004], 2 s, 1 ts, 88.5 loss
 - exp_12_batch_size=8_dense_1=5.0_epochs=4_num_layers={'den_optim=adam:	TERMINATED [pid=26042], 2 s, 1 ts, 97.3 loss
 - exp_13_batch_size=16_dense_1=8.0_epochs=3_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26080], 2 s, 1 ts, 171 loss
 - exp_14

Created LogSyncer for /home/jma/ray_results/my_exp/exp_49_batch_size=8_dense_1=7.0_epochs=4_num_layers={'lay_optim=rmspr_2018-07-19_15-47-33dxojcbid -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/8 CPUs, 0/1 GPUs
Result logdir: /home/jma/ray_results/my_exp
TERMINATED trials:
 - exp_0_batch_size=8_dense_1=7.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25521], 2 s, 1 ts, 412 loss
 - exp_10_batch_size=8_dense_1=6.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=25966], 2 s, 1 ts, 258 loss
 - exp_11_batch_size=8_dense_1=10.0_epochs=4_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26004], 2 s, 1 ts, 88.5 loss
 - exp_12_batch_size=8_dense_1=5.0_epochs=4_num_layers={'den_optim=adam:	TERMINATED [pid=26042], 2 s, 1 ts, 97.3 loss
 - exp_13_batch_size=16_dense_1=8.0_epochs=3_num_layers={'lay_optim=rmspr:	TERMINATED [pid=26080], 2 s, 1 ts, 171 loss
 - exp_14_batch_size=32_dense_1=5.0_epochs=1_num_layers={'den_optim=adam:	TERMINATED [pid=26118], 1 s, 1 ts,

'0:00:01.936235'

In [9]:
trials[0].last_result.config['batch_size']
trials[0].last_result.mean_loss

16